In [1]:
#python install pillow
from PIL import Image
import numpy as np
from Utils import normalize, denormalize
import tensorflow as tf

sub_size = 96

###-----------在此更改所需的图片路径-----------###
file_path = "C:\\Users\\Administrator\\Desktop\\"
image_name = '123.jpg'
imgs_path = file_path + image_name
###------------------------------------------###


model_path = '.\\Trained_model\\gen_model3000.h5'
output_path = '.\\application\\'

In [2]:
#分割图片
def cut_image(img_path):
    image = Image.open(img_path)
    width, height = image.size
    count_x = width // sub_size
    count_y = height // sub_size
    
    box_list = []
    # (left, upper, right, lower)
    for j in range(count_y):
        for i in range(count_x):
            box = (i*sub_size, j*sub_size, (i+1)*sub_size, (j+1)*sub_size)
            box_list.append(box)
    image_list = [image.crop(box) for box in box_list]
    image_list_array = []
    for img in image_list:
        image = normalize(np.array(img).astype(np.float64))
        image_list_array.append(image)
    
    return (count_x, count_y, image_list_array)

In [3]:
def load_generator(path):
    Gen = tf.keras.models.load_model(path, compile=False)
    return Gen

def gen_new_one(model_path, imgs_path, output_path):
    gen = load_generator(model_path)
    (x, y, image_list) = cut_image(imgs_path)
    gen = load_generator(model_path)
    image_gen_list = []
    for i in range(len(image_list)):
        image_gen_list.append(denormalize(gen.predict(np.array([image_list[i]]))))
    rows = []
    for j in range(y):
        row = image_gen_list[j*x]
        for i in range(1,x):
            row = np.concatenate((row,image_gen_list[j*x+i]),axis=2)
        rows.append(row)
    column = rows[0]
    for j in range(1,y):
        column = np.concatenate((column,rows[j]),axis=1)
    
    Image.fromarray(np.uint8(column[0])).save(output_path +'gen_' + image_name)

In [4]:
if __name__ == "__main__":
    gen_new_one(model_path, imgs_path, output_path)